In [216]:
#populating random user ratings - 0 represents item not rated 
#Limitation - at least one item must be rated to be a candidate for recommendation for collaborative filtering
import math
import numpy as np
import pandas as pd

items = 5
users = 10
a = np.random.rand(users,items)
df = pd.DataFrame(a)
for i in range(users):
    for j in range(items):
        df.set_value(i,j,20*df.get_value(i,j))
        df.set_value(i,j,int(math.ceil(df.get_value(i,j))))
        if df.get_value(i,j)>10:
            df.set_value(i,j,0)
df.rename(columns = {0: "A", 1: "B",2:"C",3:"D",4:"E"})



,A,B,C,D,E
0,0.0,0.0,1.0,7.0,4.0
1,0.0,6.0,0.0,7.0,0.0
2,0.0,0.0,0.0,9.0,0.0
3,3.0,0.0,0.0,6.0,0.0
4,0.0,7.0,0.0,8.0,4.0
5,8.0,0.0,7.0,0.0,0.0
6,0.0,3.0,0.0,0.0,0.0
7,4.0,0.0,0.0,0.0,0.0
8,6.0,10.0,0.0,10.0,0.0
9,0.0,0.0,0.0,0.0,6.0


In [217]:
#edge weights in dimension 0
b = [[0 for x in range(users)] for y in range(users)]
from scipy import spatial 
for i in range(users):
    for j in range(users):
        b[i][j] = spatial.distance.cosine(df.loc[[i]],df.loc[[j]])*100
df1 = pd.DataFrame(b)
print (df1)

              0           1           2             3           4  \
0  2.220446e-14   34.579381   13.835956  2.293254e+01   21.969269   
1  3.457938e+01    0.000000   24.074340  3.209002e+01    6.411657   
2  1.383596e+01   24.074340    0.000000  1.055728e+01   29.563927   
3  2.293254e+01   32.090025   10.557281  1.110223e-14   37.000062   
4  2.196927e+01    6.411657   29.563927  3.700006e+01    0.000000   
5  9.189437e+01  100.000000  100.000000  6.634375e+01  100.000000   
6  1.000000e+02   34.920863  100.000000  1.000000e+02   38.368437   
7  1.000000e+02  100.000000  100.000000  5.527864e+01  100.000000   
8  4.391198e+01    8.213695   34.905545  2.431107e+01   14.031292   
9  5.076340e+01  100.000000  100.000000  1.000000e+02   64.781964   

              5           6           7             8           9  
0  9.189437e+01  100.000000  100.000000  4.391198e+01   50.763404  
1  1.000000e+02   34.920863  100.000000  8.213695e+00  100.000000  
2  1.000000e+02  100.000000  100.000

In [218]:
#edge weights in dimension 1 
#uncomment following snipet if amazon's item-to-item algorithm applied
# for i in range(users):
#     for j in range(items):
#         if(df.get_value(i,j)>0):#assuming every rated item is bought
#             df.set_value(i,j,1)
# print (df)
c = [[0 for x in range(items)] for y in range(items)]
from scipy import spatial 
for i in range(items):
    for j in range(items):
        c[i][j] = spatial.distance.cosine(df[i],df[j])*100
df2 = pd.DataFrame(c)
print (df2)


              0           1             2          3           4
0  1.110223e-14   61.470322  2.916498e+01  64.163962  100.000000
1  6.147032e+01    0.000000  1.000000e+02  26.979514   75.621740
2  2.916498e+01  100.000000  1.110223e-14  94.914971   93.140057
3  6.416396e+01   26.979514  9.491497e+01   0.000000   62.625345
4  1.000000e+02   75.621740  9.314006e+01  62.625345    0.000000


In [219]:
#set parameter phi in dimension 0
#cosine distance threshold for two users
sum = 0
for i in range(users):
    print (df1.sort_values(by = i)[i].values[0:3])
    sum = sum + (df1.sort_values(by = i)[i].values[2])
sum = sum/users
print ("\n")
print (sum)

phi = sum

[  2.22044605e-14   1.38359563e+01   2.19692693e+01]
[ 0.          6.41165708  8.2136948 ]
[  0.          10.5572809   13.83595631]
[  1.11022302e-14   1.05572809e+01   2.29325364e+01]
[  0.           6.41165708  14.0312915 ]
[  1.11022302e-14   2.47423305e+01   6.63437470e+01]
[  0.          34.90554451  34.92086265]
[  0.          24.74233053  55.27864045]
[ -2.22044605e-14   8.21369480e+00   1.40312915e+01]
[  0.          50.76340361  64.78196375]


31.6339253801


In [220]:
#set parameter mu in dimension 1 
#cosine distance threshold for two items
sum =0
for i in range(items):
    print (df2.sort_values(by = i)[i].values[0:3])
    sum = sum + (df2.sort_values(by = i)[i].values[2])
sum = sum/items
print ("\n")
print (sum)

mu = sum
    

[  1.11022302e-14   2.91649804e+01   6.14703221e+01]
[  0.          26.9795138   61.47032206]
[  1.11022302e-14   2.91649804e+01   9.31400566e+01]
[  0.          26.9795138   62.62534529]
[  0.          62.62534529  75.62173969]


70.8655571387


In [221]:
#populate missing ratings 
l1 = []
u = 0
for i in range(users):
    if df1[i][u] < phi:
       l1.append(i)
print ("Users most similar to ",u,"are ",l1)

print (df)
df3 = df
for i in range(items):
    if (df[i][u]==0):
        l2 = [] #list of items closest to ith item
        for j in range(items):
            if df2[j][i] < mu:
                l2.append(j)
        print ("Items closest to item ",i, "are ",l2)
        edge_weight = 0
        count = 0
        for a in l1:
            for b in l2:
                edge_weight = edge_weight + df[b][a]
                if (df[b][a]!=0):
                    count = count + 1
        edge_weight = edge_weight/count 
        df3[i][u] = edge_weight
print (df3)

Users most similar to  0 are  [0, 2, 3, 4]
     0     1    2     3    4
0  0.0   0.0  1.0   7.0  4.0
1  0.0   6.0  0.0   7.0  0.0
2  0.0   0.0  0.0   9.0  0.0
3  3.0   0.0  0.0   6.0  0.0
4  0.0   7.0  0.0   8.0  4.0
5  8.0   0.0  7.0   0.0  0.0
6  0.0   3.0  0.0   0.0  0.0
7  4.0   0.0  0.0   0.0  0.0
8  6.0  10.0  0.0  10.0  0.0
9  0.0   0.0  0.0   0.0  6.0
Items closest to item  0 are  [0, 1, 2, 3]
Items closest to item  1 are  [0, 1, 3]
          0         1    2     3    4
0  5.857143   6.55102  1.0   7.0  4.0
1  0.000000   6.00000  0.0   7.0  0.0
2  0.000000   0.00000  0.0   9.0  0.0
3  3.000000   0.00000  0.0   6.0  0.0
4  0.000000   7.00000  0.0   8.0  4.0
5  8.000000   0.00000  7.0   0.0  0.0
6  0.000000   3.00000  0.0   0.0  0.0
7  4.000000   0.00000  0.0   0.0  0.0
8  6.000000  10.00000  0.0  10.0  0.0
9  0.000000   0.00000  0.0   0.0  6.0
